In [2]:
import numpy as np

In [7]:
arr1 = np.array([1, 3, 5, 8])
arr2 = np.array([1, 4, 5, 9])
arr3 = np.array([9, 5, 3, 2])

In [13]:
true_me = np.isin(arr1, arr3, invert=True)
true_me

array([ True, False, False,  True])

In [14]:
noob = "noob"
noob1 = "noob1"

In [17]:
if noob == "noob":
    print(noob)
elif noob1 == "noob1":
    print(noob1)

noob


In [18]:
y_true = [-12.6629  -12.0921  -11.712   -10.4558  -11.4405  -11.4212  -11.4503
 -11.6368  -11.4982  -11.7684  -11.7604  -10.7494  -10.8044  -11.2497
 -11.4895  -12.0323  -11.2327  -11.4158  -11.0029  -11.1425  -11.208
 -10.7851  -10.963   -11.3184  -10.9386  -11.0161  -10.8287  -11.094
 -10.8468  -10.7572  -10.8458  -11.1544  -10.7044  -10.9825  -10.8618
 -10.3827  -10.6054  -10.6133  -10.0682  -10.7319  -10.7359  -10.5868
 -10.3491  -10.5518  -10.6733  -10.8114  -10.5383  -10.5661  -10.7406
 -10.3962  -10.2192  -10.0984   -9.88886 -10.1059  -10.7421  -10.5682
 -10.6166  -10.4829  -10.7092  -10.3909  -10.4256   -9.81056 -10.5455
 -10.3011  -10.3162  -10.5463  -10.9348   -9.76958  -9.90383 -10.3181
 -10.7397  -10.2695  -10.4968  -10.3868  -10.4434  -10.2863  -10.545
 -10.1776  -10.4174  -10.3632  -10.4561  -10.5156  -10.3458  -10.0995
 -10.0139   -9.97559  -9.73212 -10.1953  -10.0452  -10.1555   -9.78553
 -10.428   -10.1544  -10.2542  -10.7282  -10.2034  -10.1199   -9.62462
 -10.25    -10.227  ]